### CPT Project

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2

# used to calculate AUROC/accuracy
from sklearn import metrics

%matplotlib inline

In [2]:
# create a database connection
sqluser = 'mimic' 
dbname = 'mimic'
schema_name = 'mimiciii'

# Connect to local postgres version of mimic
con = psycopg2.connect(dbname=dbname, user=sqluser)
cur = con.cursor()

In [3]:
cur.execute('SET search_path to ' + schema_name)
query = \
"""
SELECT pe.subject_id, pe.hadm_id, pe.icustay_id, pe.itemid, -- ICUSTAY_ID
       pr.icd9_code, icd.short_title, -- ICD9_CODES
       pe.cgid, c.label, -- CGIDS
       -- cpt.cpt_cd, cpt.cpt_number, -- CPT CODES
       p.dob, p.dod, p.expire_flag, -- YEARS SURVIVED AFTER DISCHARGE
       a.admittime, a.dischtime, a.deathtime, a.hospital_expire_flag
       
FROM PROCEDUREEVENTS_MV pe
JOIN ADMISSIONS a
    ON pe.hadm_id = a.hadm_id    
JOIN PATIENTS p
    ON pe.subject_id = p.subject_id
JOIN PROCEDURES_ICD pr 
    ON pe.subject_id = pr.subject_id
JOIN CAREGIVERS c
    ON pe.cgid = c.cgid
JOIN D_ICD_PROCEDURES icd
    ON pr.icd9_code = icd.icd9_code
--JOIN CPTEVENTS cpt
--    ON pe.hadm_id = cpt.hadm_id
"""
data = pd.read_sql_query(query,con)

print "Table Dimensions:", data.shape[0], "rows and ", data.shape[1], "columns"

data.head()

Table Dimensions: 2661091 rows and  15 columns


,subject_id,hadm_id,icustay_id,itemid,icd9_code,short_title,cgid,label,dob,dod,expire_flag,admittime,dischtime,deathtime,hospital_expire_flag
0,18846,186928,249733.0,224275,4513,Sm bowel endoscopy NEC,20706,RN,2074-02-01,None,0,2136-06-07 19:47:00,2136-06-15 13:30:00,None,0
1,18846,186928,249733.0,224275,9915,Parent infus nutrit sub,20706,RN,2074-02-01,None,0,2136-06-07 19:47:00,2136-06-15 13:30:00,None,0
2,18846,186928,249733.0,224275,5187,Endosc inser stent bile,20706,RN,2074-02-01,None,0,2136-06-07 19:47:00,2136-06-15 13:30:00,None,0
3,18846,186928,249733.0,224275,5188,Endosc remove bile stone,20706,RN,2074-02-01,None,0,2136-06-07 19:47:00,2136-06-15 13:30:00,None,0
4,18846,186928,249733.0,224275,5184,Endosc dilation ampulla,20706,RN,2074-02-01,None,0,2136-06-07 19:47:00,2136-06-15 13:30:00,None,0


In [4]:
cur.execute('SET search_path to ' + schema_name)
query = \
"""
SELECT pe.subject_id, pe.hadm_id, pe.icustay_id, pe.itemid, -- ICUSTAY_ID
       pr.icd9_code, icd.short_title, -- ICD9_CODES
       pe.cgid, c.label, -- CGIDS
       cpt.cpt_cd, cpt.cpt_number, -- CPT CODES
       p.dob, p.dod, p.expire_flag, -- YEARS SURVIVED AFTER DISCHARGE
       a.admittime, a.dischtime, a.deathtime, a.hospital_expire_flag
       
FROM PROCEDUREEVENTS_MV pe
JOIN ADMISSIONS a
    ON pe.hadm_id = a.hadm_id    
JOIN PATIENTS p
    ON pe.subject_id = p.subject_id
JOIN PROCEDURES_ICD pr 
    ON pe.subject_id = pr.subject_id
JOIN CAREGIVERS c
    ON pe.cgid = c.cgid
JOIN D_ICD_PROCEDURES icd
    ON pr.icd9_code = icd.icd9_code
JOIN CPTEVENTS cpt
    ON pe.hadm_id = cpt.hadm_id
LIMIT 100
"""
data = pd.read_sql_query(query,con)

print "Table Dimensions:", data.shape[0], "rows and ", data.shape[1], "columns"

data.head()

Table Dimensions: 100 rows and  17 columns


,subject_id,hadm_id,icustay_id,itemid,icd9_code,short_title,cgid,label,cpt_cd,cpt_number,dob,dod,expire_flag,admittime,dischtime,deathtime,hospital_expire_flag
0,29765,173513,None,225451,311,Temporary tracheostomy,19348,RN,90935,90935,2098-04-26,2167-10-25,1,2166-12-31 18:11:00,2167-03-02 17:00:00,None,0
1,29765,173513,None,225451,4311,Percu endosc gastrostomy,19348,RN,90935,90935,2098-04-26,2167-10-25,1,2166-12-31 18:11:00,2167-03-02 17:00:00,None,0
2,29765,173513,None,225451,966,Entral infus nutrit sub,19348,RN,90935,90935,2098-04-26,2167-10-25,1,2166-12-31 18:11:00,2167-03-02 17:00:00,None,0
3,29765,173513,None,225451,3323,Other bronchoscopy,19348,RN,90935,90935,2098-04-26,2167-10-25,1,2166-12-31 18:11:00,2167-03-02 17:00:00,None,0
4,29765,173513,None,225451,9604,Insert endotracheal tube,19348,RN,90935,90935,2098-04-26,2167-10-25,1,2166-12-31 18:11:00,2167-03-02 17:00:00,None,0
